In [4]:
import pandas as pd
import glob
import random
import base64
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
from IPython.display import HTML
import io

pd.set_option('display.max_colwidth', None)

In [14]:

def image_base64(im):
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    return f'<img src="data:image/jpeg;base64,{image_base64(im)}">'

def image_formatter2(im):
    return f'<img src="data:image/jpeg;base64,{im}">'

cifar_class_map = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck',
}

def label_df(df):
    df['actual'] = df['actual'].apply(lambda x:cifar_class_map[x])
    df['prediction']=df['prediction'].apply(lambda x:cifar_class_map[x])
    return df

# HTML(df.to_html(formatters={'image': image_formatter2}, escape=False))

In [40]:
baseline1_adv = label_df(pd.read_csv("./dev_adv_dataset_results.csv"))
baseline1_adv['adv_detection_result']=np.all(baseline1_adv[['c1','c2','c3','c4']],axis=1)
baseline1_real = label_df(pd.read_csv("./dev_real_dataset_results.csv"))
baseline1_real['adv_detection_result']=np.all(baseline1_real[['c1','c2','c3','c4']],axis=1)

countergan_adv = pd.read_csv("./counter_gan_adv_pred.csv")
countergan_real = pd.read_csv("./counter_gan_real_pred.csv")
countergan_adv = countergan_adv.drop(['Unnamed: 0'], axis=1)
countergan_real = countergan_real.drop(['Unnamed: 0'], axis=1)

defense_gan_real = label_df(pd.read_csv("./defensegan_dev_real_dataset_results.csv"))
defense_gan_adv = label_df(pd.read_csv("./defensegan_dev_adv_dataset_results.csv"))

# Baseline 1 results

In [42]:
total_real = len(baseline1_real)
total_adv = len(baseline1_adv)
adversarial_detection_score = baseline1_adv.adv_detection_result.sum()/total_adv # TP/(TP+FN)since we know all images are adv in baseline1_adv
adversarial_false_positives = baseline1_real.adv_detection_result.sum()/total_real #FP/(FP+TN)
# (baseline1_adv.actual != baseline1_adv.prediction and baseline1_adv.result).sum()
print("Adversarial Detection Score:",adversarial_detection_score)
print("Falsely Detected Real Image Score:", adversarial_false_positives)
real_html = baseline1_real.to_html(formatters={'image': image_formatter2}, escape=False)
adv_html = baseline1_adv.to_html(formatters={'image': image_formatter2}, escape=False)
with open("baseline1_realimages_visualization.html","w") as f:
    f.write(real_html)
with open("baseline1_advimages_visualization.html","w") as f:
    f.write(adv_html)

Adversarial Detection Score: 0.9689562087582484
Falsely Detected Real Image Score: 0.0


# Baseline 2(DefenseGAN Results)

In [46]:
total_real = len(defense_gan_real)
total_adv = len(defense_gan_adv)
adversarial_detection_score = (defense_gan_adv.actual == defense_gan_adv.prediction).sum()/total_adv # TP/(TP+FN)since we know all images are adv in baseline1_adv
adversarial_false_positives =  (defense_gan_real.actual == defense_gan_real.prediction).sum()/total_real #FP/(FP+TN)
# (baseline1_adv.actual != baseline1_adv.prediction and baseline1_adv.result).sum()
print("Adversarial Detection Score:",adversarial_detection_score)
print("Falsely Detected Real Image Score:", adversarial_false_positives)
real_html = baseline1_real.to_html(formatters={'image': image_formatter2}, escape=False)
adv_html = baseline1_adv.to_html(formatters={'image': image_formatter2}, escape=False)
with open("defensegan_realimages_visualization.html","w") as f:
    f.write(real_html)
with open("defensegan_advimages_visualization.html","w") as f:
    f.write(adv_html)

Adversarial Detection Score: 0.07739038563127311
Falsely Detected Real Image Score: 0.0808


# Counter GAN Results

In [49]:
countergan_adv.head()
total_real = len(countergan_real)
total_adv = len(countergan_adv)
without_defense_adv_score = (countergan_adv.gt_value == countergan_adv.classifier).sum()/total_adv
without_defense_real_score = (countergan_real.gt_value == countergan_real.classifier).sum()/total_real
adversarial_detection_score = (countergan_adv.gt_value == countergan_adv.defense_pred).sum()/total_adv # TP/(TP+FN)
adversarial_false_positives =  (countergan_real.gt_value == countergan_real.defense_pred).sum()/total_real #FP/(FP+TN)
print("Without Defense:")
print("Classifier Score on Adv dataset:",without_defense_adv_score)
print("Classifier Score on Real dataset:", without_defense_real_score)

print("\nWith Defense:")
print("Classifier Score on Adv dataset with defense:",adversarial_detection_score)
print("Classifier Score on Real dataset with defense:", adversarial_false_positives)

Without Defense:
Classifier Score on Adv dataset: 0.010323660714285714
Classifier Score on Real dataset: 0.9221833881578947

With Defense:
Classifier Score on Adv dataset with defense: 0.11356026785714286
Classifier Score on Real dataset with defense: 0.28155838815789475
